In [16]:
!pwd

/workspace/code/gym-continuousDoubleAuction_agent_based_0/gym_continuousDoubleAuction


In [20]:
%run ../gym_continuousDoubleAuction/test/test_accounting_system.py

FE....
ERROR: test_maker_modify_then_fill (__main__.TestMARLAccounting)
Scenario:
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/workspace/code/gym-continuousDoubleAuction_agent_based_0/gym_continuousDoubleAuction/test/test_accounting_system.py", line 343, in test_maker_modify_then_fill
    self.acc.modify_cash_transfer(new_quote, order)
  File "/workspace/code/gym-continuousDoubleAuction_agent_based_0/gym_continuousDoubleAuction/envs/account/cash_processor.py", line 74, in modify_cash_transfer
    order_val = (order.price) * (order.quantity)
AttributeError: 'dict' object has no attribute 'price'

FAIL: test_maker_close_long (__main__.TestMARLAccounting)
Scenario: Agent is Long. Places Sell Limit to exit.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/workspace/code/gym-continuousDoubleAuction_agent_based_0/gym_continuousDoubleAuction/test/test_account

SystemExit: True

In [19]:
# %run ../analyze_unused.py

In [ ]:
# %run ../gym_continuousDoubleAuction/inspect_latest_episode.py

In [ ]:
# %run ../gym_continuousDoubleAuction/test/test_cda_nsp.py

In [ ]:
# %run ../gym_continuousDoubleAuction/reproduce_bug.py